In [4]:
import cv2
import numpy as np
import os

In [5]:
face_cascade = cv2.CascadeClassifier("cascades/haarcascade_frontalface_default.xml")

In [6]:
faces = []
face_labels = []

base_dir = "14-celebrity-faces-dataset/data/val"

for face_label in os.listdir(base_dir):
    face_img_base_dir = f"{base_dir}/{face_label}"
    for face_img_path in os.listdir(face_img_base_dir):

        img = cv2.imread(f"{face_img_base_dir}/{face_img_path}")

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        faces_rect = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)

        for (x, y, w, h) in faces_rect:
            face = gray[y:y+h, x:x+w]
            faces.append(face)
            face_labels.append(face_label)

In [7]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

face_labels_encoded = encoder.fit_transform(face_labels)

In [8]:
recognizer = cv2.face.LBPHFaceRecognizer_create()

recognizer.train(faces, np.array(face_labels_encoded))

In [12]:
img_index = 0
for test_img_path in os.listdir("test-faces"):
    test_img = cv2.imread(f"test-faces/{test_img_path}")
    gray_test = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    test_faces_rect = face_cascade.detectMultiScale(gray_test, scaleFactor=1.2, minNeighbors=5)

    for (x, y, w, h) in test_faces_rect:
        test_face = gray_test[y:y+h, x:x+w]
        
        label, confidence = recognizer.predict(test_face)

        label_decoded = encoder.inverse_transform([label])

        cv2.rectangle(test_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(test_img, label_decoded[0], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        cv2.imshow(f"Result_{img_index}", test_img)
        img_index += 1

cv2.waitKey(0)
cv2.destroyAllWindows()